# 3.0 CITE-seq ADT

In [1]:
from clustergrammer2 import net
df = {}

clustergrammer2 backend version 0.2.7


In [32]:
import numpy as np
import pandas as pd
import gene_exp_10x
from copy import deepcopy
import json_scripts
import clustergrammer_groupby as cby

In [16]:
cell_dict = json_scripts.load_to_dict('../data/CITE-seq_data/human_mouse_cell_dictionary.json')

### Load ADT Data
Load ADT data, arcsinh transform the ADT levels, then Z-score ADT levels across cells.

### Drop CCR5, CCR7, and CD10 (following Seurat tutorial)

In [22]:
df['adt-ini'] = pd.read_csv('../data/CITE-seq_data/GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv', index_col=0)
df['adt'] = np.arcsinh(df['adt-ini']/5)

# drop markers with low "poor enrichments"
df['adt'] = df['adt'].drop(['CCR5', 'CCR7', 'CD10'])

# keep only human cells
df['adt'] = df['adt'][cell_dict['human-cells']]

net.load_df(df['adt'])
net.normalize(axis='row', norm_type='zscore')
df['adt-z'] = net.export_df()

df['adt'].shape

(10, 7339)

# Visualize ADT Levels (Z-scored)

In [23]:
cols = df['adt-z'].columns.tolist()
new_cols = [(x,) for x in cols]
df['adt-z-cat'] = deepcopy(df['adt-z'])
df['adt-z-cat'].columns = new_cols

In [35]:
net.load_df(df['adt-z-cat'])
net.cluster()
net.dendro_cats(axis='col', dendro_level=2)
df['adt-groups'] = net.export_df()
net.cluster()
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD3", "ini": 10, "clust": 1, "rank": 9, "rankvar": 0, "group":…

In [36]:
cols = df['adt-groups'].columns.tolist()

In [38]:
group_info = [x[1] for x in cols]

In [44]:
ser_groups = pd.Series(group_info)


(7339,)

In [45]:
group_val_counts = ser_groups.value_counts()
print(group_val_counts.shape)
group_val_counts

(30,)


Group 2: cat-4     2529
Group 2: cat-19    1455
Group 2: cat-26     891
Group 2: cat-20     623
Group 2: cat-10     410
Group 2: cat-29     305
Group 2: cat-25     267
Group 2: cat-18     169
Group 2: cat-24     152
Group 2: cat-16     131
Group 2: cat-7      131
Group 2: cat-23      51
Group 2: cat-13      50
Group 2: cat-27      49
Group 2: cat-1       32
Group 2: cat-8       20
Group 2: cat-9       13
Group 2: cat-15      10
Group 2: cat-2        9
Group 2: cat-5        8
Group 2: cat-17       7
Group 2: cat-11       5
Group 2: cat-22       5
Group 2: cat-6        4
Group 2: cat-12       4
Group 2: cat-3        3
Group 2: cat-21       2
Group 2: cat-28       2
Group 2: cat-14       1
Group 2: cat-30       1
dtype: int64

In [47]:
keep_group_val_counts = group_val_counts[group_val_counts >= 15]
print(keep_group_val_counts.shape)
keep_group_val_counts

(16,)


Group 2: cat-4     2529
Group 2: cat-19    1455
Group 2: cat-26     891
Group 2: cat-20     623
Group 2: cat-10     410
Group 2: cat-29     305
Group 2: cat-25     267
Group 2: cat-18     169
Group 2: cat-24     152
Group 2: cat-16     131
Group 2: cat-7      131
Group 2: cat-23      51
Group 2: cat-13      50
Group 2: cat-27      49
Group 2: cat-1       32
Group 2: cat-8       20
dtype: int64

In [33]:
df['cat-sig'], keep_genes_dict, df_gene_pval, all_fold_info = cby.generate_signatures(df['adt-groups'], category_level='Group 2')

/Users/nickfernandez/anaconda3/envs/py36lab/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3146: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/nickfernandez/anaconda3/envs/py36lab/lib/python3.6/site-packages/numpy/core/_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/nickfernandez/anaconda3/envs/py36lab/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda3/envs/py36lab/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/nickfernandez/anaconda3/envs/py36lab/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [34]:
net.load_df(df['cat-sig'])
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CD11c", "ini": 10, "clust": 5, "rank": 6, "rankvar": 5, "group…

In [13]:
import zipfile